In [ ]:
!pip install openai

In [5]:
from datasets import load_dataset, Dataset
import openai
import csv
dataset1 = load_dataset('json', data_files='sub_datasets/sub_dataset_1.json', split=None)
dataset2 = load_dataset('json', data_files='sub_datasets/sub_dataset_2.json', split=None)
dataset3 = load_dataset('json', data_files='sub_datasets/sub_dataset_3.json', split=None)
dataset4 = load_dataset('json', data_files='sub_datasets/sub_dataset_4.json', split=None)
dataset5 = load_dataset('json', data_files='sub_datasets/sub_dataset_5.json', split=None)

In [ ]:
!pip install zhipuai

In [33]:
from zhipuai import ZhipuAI
# 创建 OpenAI 客户端实例
# GPT-3.5-turbo
client = ZhipuAI(api_key="15bb7a5e8a7696d57023e25e5f9a47e2.0gEPh5r0RMNz0vIN")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"you are a talented mathematician"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="glm-4",
            messages=conversation_history,
            stream=True
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response
    
GPT35res = []

print('在测试集上检查GLM-4表现')
for i in list(range(500)):
    print('%'*30, i, '%'*30)
    item = dataset5['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text']+'请输出你的最终答案并保证这个答案是最简形式，不需要任何其他内容，只输出答案本身']
    answer = get_response(question)
    print('glm4答案', answer)
    GPT35res.append([item['text'], item['answer_expressions'], answer])
    

在测试集上检查GLM-4表现
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 0 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知直线$l$: $y=k x+1$与椭圆$\frac{x^{2}}{2}+y^{2}=1$交于$M$、$N$两点，且$|M N|=\frac{4 \sqrt{2}}{3}$，则$k$=?
正确答案 pm*1
glm4答案 $k = ±1$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 1 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 直线$y=x+b$与曲线$y=\sqrt{x^{2}-1}$有且仅有一个公共点，则$b$的取值范围是?
正确答案 [-1, 0) + [1, +oo) 
glm4答案 $b$的取值范围是 $(-1, 1) ∪ (\sqrt{2}, +∞)$。
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 2 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知双曲线$\frac{x^{2}}{4}-\frac{y^{2}}{b^{2}}=1(b>0)$的离心率为$2$，则它的一焦点到其中一条渐近线的距离为?
正确答案 2*sqrt(3)
glm4答案 1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 3 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 设$F_{1}$、$F_{2}$分别是双曲线$x^{2}-\frac{y^{2}}{9}=1$的左、右焦点，若点$P$在双曲线上，且$\overrightarrow{PF_{1}} \cdot  \overrightarrow{PF_{2}}=0$,则$|P F_{1}+P F_{2}|$=?
正确答案 2*sqrt(10)
glm4答案 8
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 4 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知$F_{1}$、$F_{2}$分别为双曲线$\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0, b>0)$的左、右焦点，$P$为双曲线右支上任意一点，若$\frac{|PF_{1}|^{2}}{|PF_{2}|}$的最

IndexError: Invalid key: 69 is out of bounds for size 69

In [34]:
# 打开一个CSV文件进行写入
with open('glm4_result5_short.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 循环写入数据
    for row in GPT35res:
        writer.writerow(row)

print('数据已成功写入CSV文件.')

数据已成功写入CSV文件.


In [6]:
# 1
from openai import OpenAI

# 替换为您的 API 令牌
API_TOKEN = 'sk-570d2f34ff1dc964eb0ae0e827a69d8a'

# 创建 OpenAI 客户端
client = OpenAI(
    api_key=API_TOKEN,
    base_url="https://api.atomecho.cn/v1",  # Llama 社区的 API 地址
)

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"你是一个圆锥曲线很厉害的数学家"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="Llama3-Chinese-8B-Instruct",
            messages=conversation_history,
            stream=True,
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response

In [7]:
# 2
import time
batch_size = 10
total_requests = 500
ansall = []
start_time = time.time()

for i in range(500):
    if i % batch_size == 0 and i != 0:    
        elapsed_time = time.time() - start_time
        if elapsed_time < 50:
            time_to_wait = 50 - elapsed_time
            print(f"Waiting for {time_to_wait:.2f} seconds...")
            time.sleep(time_to_wait)
        start_time = time.time()  # 重置计时器
        
    print("-" * 20, i)
    item = dataset2['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text'],
                "请直接给出上面问题的答案，不需要分析过程和说明文字"]
    answer = get_response(question)
    print('llama答案', answer)

    ansall.append([i, answer, item['answer_expressions']])

-------------------- 0
问题 已知$c$是双曲线$C$:$\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1,(a>0, b>0)$的半焦距，离心率为$e$，则$\frac{1}{e}+\frac{b}{c}$的最大值是?
正确答案 sqrt(2)
llama答案 1/e+a/[（1+c²e²）(ac)]
-------------------- 1
问题 已知点$P$在抛物线$x^{2}=4 y$上运动，$F$为抛物线的焦点，点$A$的坐标为$(2,3)$，则$P A+P F$取得最小值时点$P$的坐标为?
正确答案 (2, 1)
llama答案 点$P$的坐标为$\boxed{(0,\frac{-5}{24})}$。
-------------------- 2
问题 已知$M$是双曲线$\frac{x^{2}}{64}-\frac{y^{2}}{36}=1$上一点，$F_{1}$、$F_{2}$是双曲线的左、右焦点，且$|M F_{1}|=17$，则$|M F_{2}|$=?
正确答案 33
llama答案 答案为 **10** 。
-------------------- 3
问题 已知椭圆$C$的离心率$e=\frac{1}{3}$，左右焦点分别为$F_{1}$、$F_{2}$、$P$为椭圆$C$上一动点，则$\frac{|P F_{1}|}{|P F_{2}|}$的取值范围为?
正确答案 [1/2,2]
llama答案 $(0,1]$
-------------------- 4
问题 点$P$是椭圆$C$:$ \frac{x^{2}}{16}+\frac{y^{2}}{7}=1$上的一点，$F_{1}$、$F_{2}$是椭圆的两个焦点，且$\Delta P F_{1} F_{2}$的内切圆半径为$1$. 当点$P$在第一象限时，它的纵坐标为?
正确答案 7/3
llama答案 点$P$在第一象限时，其纵坐标为 $\dfrac{5}{6}\cdot\sqrt{\dfrac{3}{4}}$ 。
-------------------- 5
问题 已知点$F$为抛物线$C$: $x^{2}=2 y$的焦点，过点$F$作两条互相垂直的直线$l_{1} $, $l_{2}$，直线$l_{1}$与$C$交于$A

In [8]:
# 打开一个CSV文件进行写入
with open('llama7b_result2.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 循环写入数据
    for row in ansall:
        writer.writerow(row)

print('数据已成功写入CSV文件.')

数据已成功写入CSV文件.


In [23]:
!export DASHSCOPE_API_KEY="sk-0143ff30d9734021aec3cd09baa2256d"

'export' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [24]:
!echo $DASHSCOPE_API_KEY

$DASHSCOPE_API_KEY


In [21]:
from http import HTTPStatus
import dashscope


def call_with_messages():
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '介绍下故宫？'}]
    response = dashscope.Generation.call(
        model='baichuan2-7b-chat-v1',
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))


def call_with_prompt():
    prompt = '介绍下故宫'
    rsp = dashscope.Generation.call(model='baichuan2-7b-chat-v1',
                                    prompt=prompt)
    print(rsp)
    if rsp.status_code == HTTPStatus.OK:
        print(rsp.output)
        print(rsp.usage)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))


if __name__ == '__main__':
    type = 'message'
    if type == 'prompt':
        call_with_prompt()
    elif type == 'message':
        call_with_messages()
    else:
        print("call type not support")

Request id: a5c72fa1-5196-9d36-b542-55ff45b5fc1d, Status code: 401, error code: InvalidApiKey, error message: Invalid API-key provided.


In [12]:
# 创建 OpenAI 客户端实例
# GPT-3.5-turbo
client = openai.OpenAI(api_key="sk-dcddd419e650bc8497744f6718d61981", base_url="https://api.baichuan-ai.com/v1/assistants")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"you are a talented mathematician"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="Baichuan4",
            messages=conversation_history,
            stream=True,
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response
    
GPT35res = []

print('在测试集上检查GPT-3.5-turbo表现')
for i in list(range(34, len(dataset5['train']))):
    print('%'*30, i, '%'*30)
    item = dataset5['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text'],
                '请输出你的最终答案并保证这个答案是最简形式，不需要任何其他内容，只输出答案本身']
    answer = get_response(question)
    print('gpt答案', answer)
    GPT35res.append([item['text'], item['answer_expressions'], answer])
    

在测试集上检查GPT-3.5-turbo表现
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 34 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 直线$y=2 x+3$与曲线$y=x^{2}$相交于$A$、$B$两点，则$|A B|$=?
正确答案 4*sqrt(5)


NotFoundError: Error code: 404 - {'timestamp': '2024-06-25T04:02:17.849+00:00', 'status': 404, 'error': 'Not Found', 'path': '/v1/assistants/chat/completions'}